In [1]:
!nvidia-smi

Thu Jun 24 21:53:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 272 (delta 56), reused 38 (delta 19), pack-reused 171
Receiving objects: 100% (272/272), 79.66 KiB | 7.96 MiB/s, done.
Resolving deltas: 100% (118/118), done.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:11 https://developer.download.nvidia.com/c

In [1]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:35
🔁 Restarting kernel...


In [1]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
# The <packages> option are default blank or 'core'.  By default, we install RAPIDSAI and BlazingSQL.  The 'core' option will install only RAPIDSAI and not include BlazingSQL, 
!python rapidsai-csp-utils/colab/install_rapids.py stable

Installing RAPIDS Stable 21.06
Starting the RAPIDS+BlazingSQL install on Colab.  This will take about 15 minutes.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=11.0
    - gcsfs
    - llvmlite
    - openssl
    - python=3.7
    - rapids-blazing=21.06


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abseil-cpp-20210324.1      |       h9c3ff4c_0        1015 KB  conda-forge
    aiohttp-3.7.4.post0        |   py37h5e8e339_0         625 KB  conda-forge
    anyio-3.2.1                |   py37h89c1867_0         138 KB  conda-forge
    appdirs-1.4.4              |     pyh9f0ad1d_0      

# Comparing cuML with sklearn

In [4]:
import cudf

In [6]:
from cuml import make_regression,train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

In [10]:
n_samples=2**10
n_features=399
random_state=23


### Generating some random regression data

In [11]:
%%time

X,y=make_regression(n_samples=n_samples,n_features=n_features,random_state=random_state)
X=cudf.DataFrame(X)
y=cudf.DataFrame(y)[0]

X_cudf,X_cudf_test,y_cudf,y_cudf_test=train_test_split(X,y,test_size=0.2,random_state=random_state)

CPU times: user 2.72 s, sys: 773 ms, total: 3.49 s
Wall time: 12.8 s


In [12]:
y

0       -57.295097
1       -37.026691
2        43.880489
3      -185.610519
4        50.494812
           ...    
1019    115.639580
1020    -20.502155
1021    -12.507671
1022     49.697124
1023     92.181160
Name: 0, Length: 1024, dtype: float32

In [14]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398
0,0.817792,-0.014241,0.737288,0.306367,-0.897469,1.439516,-0.525046,-1.097936,0.140833,0.319280,-1.253977,-2.809769,0.652297,0.220317,1.451915,-0.686665,-1.261023,0.532241,-0.445225,-0.326115,0.419603,-0.273596,-1.281222,-0.569973,1.314366,-1.043378,0.249512,-0.137114,-0.055088,0.211612,-1.528601,1.846829,1.198249,1.121857,1.166460,-0.976729,0.146362,0.812326,1.000305,-0.733997,-0.895602
1,-1.436997,-0.525358,0.267310,0.663213,-0.495826,1.584335,-0.400521,0.832231,-2.127464,0.386978,-0.337680,-0.041267,0.710038,-0.421984,-0.300115,2.222483,-0.702894,-0.584604,0.029247,-0.499536,1.645055,0.472040,-1.098430,-0.556830,0.218017,-0.192175,-1.520975,1.619004,-1.026093,0.171732,-2.472025,-0.557467,-0.861660,0.520139,0.766996,2.359004,0.634321,1.119001,0.332424,-0.419821,-0.115555
2,0.679336,2.055207,-0.414042,0.435699,-1.473669,1.443924,0.029960,0.639099,0.641732,0.601448,1.228310,0.774208,0.354695,-1.125633,0.370567,-1.158106,0.688744,0.789121,-1.995733,0.158892,0.344290,0.206164,-0.099382,0.658434,0.586817,1.886490,0.129249,1.379174,-0.188714,-0.472445,0.387634,0.115131,-1.607258,-0.509717,-0.981310,0.649526,0.034318,0.301607,0.632835,-0.301301,-0.555823
3,0.143297,0.040230,1.405870,0.300032,0.136360,-0.200971,0.402918,-0.886953,-1.717241,-0.458499,-0.211931,-1.753499,-0.010218,-0.168953,-0.801636,-0.222612,-0.487848,-1.876990,0.802657,-1.239880,-0.074954,-0.796903,-0.261182,0.607311,-2.222911,0.635224,-0.841444,0.986202,-0.033170,0.016774,0.401204,-0.532619,-0.233561,1.102019,-1.725533,2.454678,-0.261703,-0.598027,-1.009561,-2.184268,-0.439046
4,-0.485583,0.070187,0.059087,0.730081,2.358043,0.081235,0.247418,-1.722037,1.493614,1.150906,1.013796,0.682789,-1.207039,-1.065786,0.600039,-0.535232,0.157339,-0.039439,0.598304,0.978133,1.263506,1.455037,-0.197175,-0.051686,0.171022,-0.990101,1.579820,-0.759008,-1.236704,0.313159,-0.858596,-1.367833,-0.410256,-1.423813,-0.295760,0.767513,0.831339,-0.882137,-0.979189,2.628245,-0.643571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,-1.480667,1.007604,0.853253,-2.090851,0.008597,1.343102,-0.410291,-1.182406,1.293637,-0.913647,-1.184150,-0.242934,0.780988,0.285580,0.341875,-0.712755,-0.139832,0.067008,-0.325720,-0.869751,1.523903,-1.330911,-0.839919,-0.227140,1.298228,-0.082544,2.424839,0.895559,0.030609,-1.397807,1.765726,1.942659,0.625097,0.089422,1.122334,0.459962,-1.235340,-0.175782,0.961186,0.292247,1.168002
1020,-1.914898,-0.617123,0.808683,-0.070810,-0.276406,-0.847877,-0.335927,-0.382082,-0.766061,-2.374678,0.455428,0.357437,-1.082309,-0.465495,0.936583,-0.651178,-0.478079,0.749894,1.570175,-0.852942,-0.485562,0.492730,0.581316,-0.782843,1.369206,1.789845,-2.139187,1.276871,-0.091550,0.506322,0.454950,-0.659323,0.633180,-0.485053,2.524066,-1.750670,-0.653676,0.573441,-1.077676,-0.140670,-0.446245
1021,-1.691522,1.651424,-0.101870,0.309039,-1.408558,1.051391,-0.216758,-1.241509,1.302452,-1.167881,0.123677,-0.430520,0.174297,1.870196,0.301073,1.309726,-0.844703,1.506899,0.127352,0.106251,0.821367,0.908946,0.906697,0.749822,-0.734872,1.166261,-0.888109,0.985368,0.663957,-0.209252,0.519432,-0.204346,-0.673408,0.053060,1.613348,-2.152904,-0.628705,-0.327652,1.243434,0.047226,0.328128
1022,1.706430,1.415768,-0.974009,-2.033604,-0.027225,-0.431392,0.260385,1.208408,-1.085110,-0.206307,-0.707619,-1.619134,1.159053,-0.919050,0.812265,-1.497981,-1.278399,1.492455,-0.286076,-0.474615,-0.621843,0.186543,-0.209510,1.578696,1.179122,0.542440,0.767310,-0.104846,-0.880705,-0.659082,0.707317,0.345078,1.109552,-0.472661,0.432990,0.592178,-1.235954,-0.886558,1.015957,-0.606937,-1.217009


In [15]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.

X_train=X_cudf.to_pandas()
X_test=X_cudf_test.to_pandas()
y_train=y_cudf.to_pandas()
y_test=y_cudf_test.to_pandas()

In [19]:
type(y_train)

pandas.core.series.Series

In [20]:
type(y_cudf)

cudf.core.series.Series

In [21]:
type(X_train)

pandas.core.frame.DataFrame

In [22]:
type(X_cudf)

cudf.core.dataframe.DataFrame

## Scikit-learn Model
### Fit, predict and evaluate

In [23]:
%%time

ols_sk=skLinearRegression(fit_intercept=True,
                          normalize=True,
                          n_jobs=-1)

ols_sk.fit(X_train,y_train)

CPU times: user 57.6 ms, sys: 30.4 ms, total: 88 ms
Wall time: 75.5 ms


In [24]:
%%time

predict_sk=ols_sk.predict(X_test)

CPU times: user 5.36 ms, sys: 0 ns, total: 5.36 ms
Wall time: 8.47 ms


In [25]:
%%time

r2_score_sk=r2_score(y_cudf_test,predict_sk)

CPU times: user 4.25 ms, sys: 864 µs, total: 5.11 ms
Wall time: 5.96 ms


## cuML Model
### Fit, predict and evaluate

In [27]:
%%time

ols_cuml=cuLinearRegression(fit_intercept=True,
                            normalize=True,
                            algorithm='eig')

ols_cuml.fit(X_cudf,y_cudf)

CPU times: user 43.4 ms, sys: 5.53 ms, total: 48.9 ms
Wall time: 171 ms


In [34]:
%%time
predict_cuml = ols_cuml.predict(X_cudf_test)

CPU times: user 23.1 ms, sys: 2.29 ms, total: 25.4 ms
Wall time: 25.5 ms


In [37]:
%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)

CPU times: user 1.29 ms, sys: 0 ns, total: 1.29 ms
Wall time: 1.3 ms


### Comparing Reuslts

In [39]:
print("R^2 score (sklearn):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

R^2 score (sklearn):  1.0
R^2 score (cuML): 1.0
